In [2]:
!rm -rf models

In [5]:
from azure.storage.blob import BlobServiceClient
import os

# Artifact Information
MODEL_ZIP_FILE = "models2023-04-01 20:01:11.zip"
AZURE_BLOB_DATA_FILE_PATH = 'Trained_models/' + MODEL_ZIP_FILE

# Azure Constants
CONTAINER_NAME = "azureml-blobstore-afaa340e-8029-4073-bfbd-01de34e364bd"
CONNECTION_STRING = "DefaultEndpointsProtocol=https;AccountName=garbagenetclas4048036826;AccountKey=EqBkL/lJUkc01dqCxs9FOrQ1qlqZBdjLYMg2kuhhs1Cx945LjUpDyAtReCNkVUThBeyF6Qnmw6re+AStO0YfxA==;EndpointSuffix=core.windows.net"

# Azure Set Up
def get_azure_blob_client():
    blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING)
    blob_client = blob_service_client.get_blob_client(container=CONTAINER_NAME, blob=AZURE_BLOB_DATA_FILE_PATH)
    return blob_client

# Getting Model Artifact
def get_model():
    blob_client = get_azure_blob_client()
    
    # Download the blob to a local file
    with open("models.zip", "wb") as my_blob:
      download_stream = blob_client.download_blob()
      my_blob.write(download_stream.readall())

get_model()

In [7]:
# Setting the required directory structure for tf-serving
!unzip 'models.zip'
!mkdir assets
!mv /mnt/batch/tasks/shared/LS_root/mounts/clusters/garbage-machine/code/Users/varunvora24/CD/assets /mnt/batch/tasks/shared/LS_root/mounts/clusters/garbage-machine/code/Users/varunvora24/CD/models/
!rm -rf "models.zip"

Archive:  models.zip
  inflating: models/saved_model.pb   
  inflating: models/fingerprint.pb   
   creating: models/assets/
  inflating: models/variables/variables.index  
  inflating: models/variables/variables.data-00000-of-00001  


In [8]:
%%writefile Dockerfile
# Docker pulls the specified image and sets it as the working image
ARG BASE_IMAGE="ubuntu:20.04"
FROM ${BASE_IMAGE}

# Allow log messages to be dumped in the stream (not buffer)
ENV PYTHONUNBUFFERED TRUE

# Install the Ubuntu dependencies and Python 3
RUN apt-get update \
    && apt-get install --no-install-recommends -y \
    ca-certificates \
    python3-dev \
    python3-distutils \
    python3-venv \
    curl \
    wget \
    unzip \
    gnupg \
    && rm -rf /var/lib/apt/lists/* \
    && cd /tmp \
    && curl -O https://bootstrap.pypa.io/get-pip.py \
    && python3 get-pip.py \
    && rm get-pip.py

# Create a new Python env and include it in the PATH
RUN python3 -m venv /home/venv
ENV PATH="/home/venv/bin:$PATH"

# Update the default system Python version to Python3/PIP3
RUN update-alternatives --install /usr/bin/python python /usr/bin/python3 1
RUN update-alternatives --install /usr/local/bin/pip pip /usr/local/bin/pip3 1

# Creates a new Ubuntu user
RUN useradd -m model-server

# Upgrades PIP before proceeding and installs setuptools
RUN pip install pip --upgrade
RUN pip install -U pip setuptools

# Installs the TensorFlow Serving requirements
RUN pip install tensorflow-cpu==2.11.0
RUN echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
RUN apt-get update && apt-get install tensorflow-model-server -y

# Downloads the TensorFlow trained model
RUN cd /home \
    && mkdir /home/saved_models/ \
    && cd /home/saved_models/ \
    && mkdir garbagenet \
    && cd garbagenet \
    && mkdir dir 1

WORKDIR /home/saved_models/

COPY models/ /home/saved_models/garbagenet/1

RUN ls --recursive /home/saved_models/

RUN pwd \
    && ls -lart \
    && cd /home/saved_models/garbagenet \
    && ls -lart

# Sets the proper rights to the /home/saved_models dir and the created Python env
RUN chown -R model-server /home/saved_models \
    && chown -R model-server /home/venv
    
# Creates a directory for the logs and sets permissions to it
RUN mkdir /home/logs \
    && chown -R model-server /home/logs

# Defines the model_path environment variable and the model name
ENV MODEL_PATH=/home/saved_models/garbagenet
ENV MODEL_NAME=garbagenet

# Expose the ports 8500 (gRPC) and 8501 (REST)
ENV GRPC_PORT=8500
ENV REST_PORT=8501
EXPOSE $GRPC_PORT $REST_PORT

# Prepare the CMD that will be run on docker run
USER model-server
CMD tensorflow_model_server --port=$GRPC_PORT --rest_api_port=$REST_PORT --model_name=$MODEL_NAME --model_base_path=$MODEL_PATH >> /home/logs/server.log

Writing Dockerfile


In [9]:
%%writefile garbage-tf-serving-image-build-push.yml
trigger:
- main

pool:
  name: Default

jobs:
- job:
  displayName: "Tf-Serving Image Build Push"
  
  steps:
  
  - task: Docker@0
    displayName: 'Build an image'
    inputs:
      azureSubscription: 'Azure subscription 1(e6d4c51e-0d82-4960-829a-196186580859)'
      azureContainerRegistry: '{"loginServer":"garbagenettrainer.azurecr.io", "id" : "/subscriptions/e6d4c51e-0d82-4960-829a-196186580859/resourceGroups/ELG5164-Garbagenet/providers/Microsoft.ContainerRegistry/registries/GarbagenetTrainer"}'
      imageName: 'garbagenet-tf-serving'
      includeLatestTag: true
      
  - task: Docker@0
    displayName: 'Push an image'
    inputs:
      azureSubscription: 'Azure subscription 1(e6d4c51e-0d82-4960-829a-196186580859)'
      azureContainerRegistry: '{"loginServer":"garbagenettrainer.azurecr.io", "id" : "/subscriptions/e6d4c51e-0d82-4960-829a-196186580859/resourceGroups/ELG5164-Garbagenet/providers/Microsoft.ContainerRegistry/registries/GarbagenetTrainer"}'
      action: 'Push an image'
      imageName: 'garbagenet-tf-serving'
      includeLatestTag: true

Writing garbage-tf-serving-image-build-push.yml


In [31]:
%%writefile garbagenet.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: "garbagenet-deployment"
spec:
  selector:
    matchLabels:
      app: "garbagenet"
  replicas: 3
  template:
    metadata:
      labels:
        app: "garbagenet"
    spec:
      imagePullSecrets:
        - name: acr-auth-secret
      containers:
        - name: "garbagenet"
          image: garbagenettrainer.azurecr.io/garbagenet-tf-serving:latest
          ports:
          - containerPort: 8500
          - containerPort: 8501
---
apiVersion: "v1"
kind: "Service"
metadata:
  name: "garbagenet-service"
  namespace: "default"
  labels:
    app: "garbagenet"
spec:
  ports:
  - name: "tensorflow-svc"
    protocol: "TCP"
    port: 8500
    targetPort: 8500
  - name: "tensorflow-rest"
    protocol: "TCP"
    port: 8501
    targetPort: 8501
  selector:
    app: "garbagenet"
  type: "LoadBalancer"

Overwriting garbagenet.yaml


In [11]:
!git lfs track models
!git add Dockerfile garbage-tf-serving-image-build-push.yml garbagenet.yaml models
!git commit -m "Preparing Deployment Files"
!git push -u --all origin

"models" already supported
[master 3b98115] Preparing Deployment Files
 1 file changed, 41 insertions(+)
 create mode 100644 garbagenet.yaml
Locking support detected on remote "origin". Consider enabling it with:
  $ git config lfs.https://varunvora24@dev.azure.com/varunvora24/ELG-5164-Garbagenet-MLOPs/_git/ELG-5164-Garbagenet-MLOPs.git/info/lfs.locksverify true
Enumerating objects: 4, done.
Counting objects: 100% (4/4), done.
Delta compression using up to 4 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 701 bytes | 33.00 KiB/s, done.
Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
remote: Analyzing objects... (3/3) (9 ms)
remote: Storing packfile... done (58 ms)
remote: Storing index... done (151 ms)
To https://dev.azure.com/varunvora24/ELG-5164-Garbagenet-MLOPs/_git/ELG-5164-Garbagenet-MLOPs
   ddb124e..3b98115  master -> master
branch 'master' set up to track 'origin/master'.


In [ ]:
!az extension add --name azure-devops
!az devops configure --defaults organization=https://dev.azure.com/varunvora24/ project=ELG-5164-Garbagenet-MLOPs
!az login --identity
!az pipelines run --name "Garbagenet-TF-Serving-Image-Build-Push"

In [1]:
!az aks create --resource-group ELG5164-Garbagenet --name garbagenet-cluster --node-count 1 --generate-ssh-keys

{| Finished ..
  "aadProfile": null,
  "addonProfiles": null,
  "agentPoolProfiles": [
    {
      "availabilityZones": null,
      "count": 1,
      "creationData": null,
      "currentOrchestratorVersion": "1.24.9",
      "enableAutoScaling": false,
      "enableEncryptionAtHost": false,
      "enableFips": false,
      "enableNodePublicIp": false,
      "enableUltraSsd": false,
      "gpuInstanceProfile": null,
      "hostGroupId": null,
      "kubeletConfig": null,
      "kubeletDiskType": "OS",
      "linuxOsConfig": null,
      "maxCount": null,
      "maxPods": 110,
      "minCount": null,
      "mode": "System",
      "name": "nodepool1",
      "nodeImageVersion": "AKSUbuntu-1804gen2containerd-202303.06.0",
      "nodeLabels": null,
      "nodePublicIpPrefixId": null,
      "nodeTaints": null,
      "orchestratorVersion": "1.24.9",
      "osDiskSizeGb": 128,
      "osDiskType": "Managed",
      "osSku": "Ubuntu",
      "osType": "Linux",
      "podSubnetId": null,
      "powerS

In [2]:
!az aks get-credentials --resource-group ELG5164-Garbagenet --name garbagenet-cluster

Merged "garbagenet-cluster" as current context in /home/azureuser/.kube/config


In [76]:
!kubectl create secret docker-registry acr-auth-secret --docker-server=garbagenettrainer.azurecr.io --docker-username=GarbagenetTrainer --docker-password=UU/P8bd7c52OianKIVaPznixIXq3cqe3zTDNG+d87f+ACRAgPBmi

secret/acr-auth-secret created


In [77]:
!pwd
!kubectl apply -f garbagenet.yaml

/mnt/batch/tasks/shared/LS_root/mounts/clusters/garbage-machine/code/Users/varunvora24/CD
deployment.apps/garbagenet-deployment created
service/garbagenet-service created


In [97]:
!kubectl get deploy
!kubectl get svc

NAME                    READY   UP-TO-DATE   AVAILABLE   AGE
garbagenet-deployment   3/3     3            3           76s
NAME                 TYPE           CLUSTER-IP    EXTERNAL-IP     PORT(S)                         AGE
garbagenet-service   LoadBalancer   10.0.15.187   20.88.106.226   8500:30484/TCP,8501:31714/TCP   77s
kubernetes           ClusterIP      10.0.0.1      <none>          443/TCP                         17h


In [98]:
!kubectl delete -f garbagenet.yaml
!kubectl delete secret acr-auth-secret

deployment.apps "garbagenet-deployment" deleted
service "garbagenet-service" deleted
